In [1]:

!pip install lightgbm==4.2.0 -i https://mirrors.aliyun.com/pypi/simple/
!pip install catboost==1.2.7 -i https://mirrors.aliyun.com/pypi/simple/
!pip install xgboost==2.0.3 -i https://mirrors.aliyun.com/pypi/simple/
!pip install joblib==1.4.2 -i https://mirrors.aliyun.com/pypi/simple/

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/



[notice] A new release of pip is available: 24.1.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Looking in indexes: https://mirrors.aliyun.com/pypi/simple/



[notice] A new release of pip is available: 24.1.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Looking in indexes: https://mirrors.aliyun.com/pypi/simple/



[notice] A new release of pip is available: 24.1.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Looking in indexes: https://mirrors.aliyun.com/pypi/simple/



[notice] A new release of pip is available: 24.1.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Ce notebook a été réalisé avec les versions 1.14.1 de scipy et 1.26.4 de numpy

In [2]:
import scipy
import numpy


print(scipy.__version__) 
print(numpy.__version__)

1.14.1
1.26.4


## Les données que l'on traitent sont énormément volumineuses donc on utilise une fonction de réduction de stockage trouvé sur internet et on importe les librairies nécessaires

In [8]:
import os
import joblib
import pandas as pd
import polars as pl
import lightgbm as lgb
import xgboost as xgb
import catboost as cbt
import numpy as np
print(np.__version__)
from joblib import Parallel, delayed


def reduce_mem_usage(self, float16_as32=True):
    # Calculate the initial memory usage of the dataframe
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:  # Iterate over each column
        col_type = df[col].dtype  # Get the column type
        if col_type != object and str(col_type) != 'category':  # Exclude object or categorical columns
            c_min, c_max = df[col].min(), df[col].max()  # Get the min and max values of the column
            if str(col_type)[:3] == 'int':  # For integer types
                # Convert to appropriate int type based on range
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:  # For floating point types
                # Convert to appropriate float type based on range
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    if float16_as32:  # Use float32 if higher precision is needed
                        df[col] = df[col].astype(np.float32)
                    else:
                        df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    
    # Calculate and print final memory usage
    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df




1.26.4


# On a 9 fichiers qui contiennent chacun des fichiers parquets correspondant le premier au dates 1-169, ensuite 170 - ... Je commence par les fusionner pour n'en avoir plus qu'un seul

In [9]:
import os
import pandas as pd
from glob import glob

# Chemin vers le répertoire principal
base_dir = "./jane-street-real-time-market-data-forecasting/train.parquetbis"

# Trouver tous les fichiers "part-0.parquet" dans les sous-dossiers "partition_id=X"
parquet_files = glob(os.path.join(base_dir, "partition_id=*", "part-0.parquet"))

# Liste pour stocker les DataFrames
dataframes = []

# Lire chaque fichier et l'ajouter à la liste
for file in parquet_files[:2]:
    print(f"Chargement du fichier : {file}")
    df = pd.read_parquet(file)  # Lire le fichier Parquet
    print("ok")
    df = reduce_mem_usage(df,False)
    dataframes.append(df)       # Ajouter le DataFrame à la liste

# Concaténer tous les DataFrames en un seul
full_data = pd.concat(dataframes, ignore_index=True)

# Sauvegarder dans un fichier Parquet combiné
output_file = os.path.join(base_dir, "combined_data.parquet")
full_data.to_parquet(output_file)

print(f"Fichier combiné sauvegardé ici : {output_file}")


Chargement du fichier : ./jane-street-real-time-market-data-forecasting/train.parquetbis\partition_id=0\part-0.parquet
ok
Memory usage of dataframe is 654.51 MB
Memory usage after optimization is: 435.72 MB
Decreased by 33.4%
Chargement du fichier : ./jane-street-real-time-market-data-forecasting/train.parquetbis\partition_id=1\part-0.parquet
ok
Memory usage of dataframe is 944.04 MB
Memory usage after optimization is: 548.24 MB
Decreased by 41.9%


C:\Users\tilia\AppData\Local\Temp\ipykernel_30976\3781661208.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  full_data = pd.concat(dataframes, ignore_index=True)


Fichier combiné sauvegardé ici : ./jane-street-real-time-market-data-forecasting/train.parquetbis\combined_data.parquet


# Petit résumé de nos données

In [6]:
import pandas as pd

# Load the provided Parquet file to inspect its contents
# file_path = './jane-street-real-time-market-data-forecasting/train.parquet/partition_id=2/part-0.parquet'
file_path = "./jane-street-real-time-market-data-forecasting/train.parquetbis/combined_data.parquet"
try:
    df = pd.read_parquet(file_path)
    print("try ok")
    df_info = df.info()  # Capture information about the dataframe structure
    df_head = df.head()  # Capture the first few rows to understand its content
except Exception as e:
    df_info = str(e)
    df_head = None

df_info, df_head

print(df["date_id"])


try ok
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4748457 entries, 0 to 4748456
Data columns (total 92 columns):
 #   Column       Dtype  
---  ------       -----  
 0   date_id      int16  
 1   time_id      int16  
 2   symbol_id    int8   
 3   weight       float16
 4   feature_00   float16
 5   feature_01   float16
 6   feature_02   float16
 7   feature_03   float16
 8   feature_04   float16
 9   feature_05   float16
 10  feature_06   float16
 11  feature_07   float16
 12  feature_08   float16
 13  feature_09   int8   
 14  feature_10   int8   
 15  feature_11   int16  
 16  feature_12   float16
 17  feature_13   float16
 18  feature_14   float16
 19  feature_15   float16
 20  feature_16   float16
 21  feature_17   float16
 22  feature_18   float16
 23  feature_19   float16
 24  feature_20   float16
 25  feature_21   float64
 26  feature_22   float16
 27  feature_23   float16
 28  feature_24   float16
 29  feature_25   float16
 30  feature_26   float64
 31  feature_27   floa

# On charge les données d'entraînement, les filtre en fonction de certaines dates, puis les sépare en ensembles d'entraînement et de validation

In [10]:
# Define the path to the input data directory
# If the local directory exists, use it; otherwise, use the Kaggle input directory
input_path = './jane-street-real-time-market-data-forecasting' #if os.path.exists('./jane-street-real-time-market-data-forecasting') #else '/kaggle/input/jane-street-real-time-market-data-forecasting/'
#file_path = './jane-street-real-time-market-data-forecasting/train.parquet/partition_id=2/part-0.parquet'
file_path = "./jane-street-real-time-market-data-forecasting/train.parquetbis/combined_data.parquet"


# Flag to determine if the script is in training mode or not
TRAINING = True

# Define the feature names based on the number of features (79 in this case)
feature_names = [f"feature_{i:02d}" for i in range(79)]




num_valid_dates = 100 # Number of validation dates to use
#skip_dates = 500 # Number of dates to skip from the beginning of the dataset (initial)
skip_dates = 10 #for the working test
N_fold = 5 # Number of folds for cross-validation

# If in training mode, load the training data
if TRAINING:
    # Load the training data from a Parquet file
    df = pd.read_parquet(file_path) 
    # df = pd.read_parquet(f'{input_path}/train.parquet2')
    
    # Reduce memory usage of the DataFrame (function not provided here)
    df = reduce_mem_usage(df, False)
    
    # Filter the DataFrame to include only dates greater than or equal to skip_dates
    df = df[df['date_id'] >= skip_dates].reset_index(drop=True)
    
    # Get unique dates from the DataFrame
    dates = df['date_id'].unique()
    
    # Define validation dates as the last `num_valid_dates` dates
    valid_dates = dates[-num_valid_dates:]
    
    # Define training dates as all dates except the last `num_valid_dates` dates
    train_dates = dates[:-num_valid_dates]
    
    # Display the last few rows of the DataFrame (for debugging purposes)
    print(df.tail())

Memory usage of dataframe is 928.34 MB
Memory usage after optimization is: 928.34 MB
Decreased by 0.0%
         date_id  time_id  symbol_id    weight  feature_00  feature_01  \
4660156      339      848         19  4.386719   -0.357910   -1.135742   
4660157      339      848         30  0.913086    0.013420   -1.095703   
4660158      339      848         33  1.225586   -0.378906   -1.521484   
4660159      339      848         34  1.267578   -0.343262   -1.547852   
4660160      339      848         38  2.267578   -0.415039   -1.260742   

         feature_02  feature_03  feature_04  feature_05  ...  feature_78  \
4660156   -0.375488    0.005455   -1.241211   -0.139771  ...   -0.118408   
4660157   -0.315430   -0.227051   -1.131836   -0.119019  ...   -0.094299   
4660158    0.209351   -0.241943   -1.596680   -0.082520  ...   -0.196045   
4660159   -0.188599    0.174194   -1.161133   -0.150146  ...   -0.221802   
4660160    0.046844   -0.117554   -1.687500   -0.096436  ...    0.178833

In [ ]:
!pip install grpcio
!pip install google
!pip install protobuf


In [11]:
# Create a directory to store the trained models
os.system('mkdir models')

# Define the path to load pre-trained models (if not in training mode)
model_path = '/kaggle/input/jsbaselinezyz'


TRAINING = True
# If in training mode, prepare validation data
if TRAINING:
    # Extract features, target, and weights for validation dates
    X_valid = df[feature_names].loc[df['date_id'].isin(valid_dates)]
    y_valid = df['responder_6'].loc[df['date_id'].isin(valid_dates)]
    w_valid = df['weight'].loc[df['date_id'].isin(valid_dates)]

# Initialize a list to store trained models
models = []

# Function to train a model or load a pre-trained model
def train(model_dict, model_name='lgb'):
    if TRAINING:
        # Select dates for training based on the fold number
        selected_dates = [date for ii, date in enumerate(train_dates) if ii % N_fold != i]
        
        # Get the model from the dictionary
        model = model_dict[model_name]
        
        # Extract features, target, and weights for the selected training dates
        X_train = df[feature_names].loc[df['date_id'].isin(selected_dates)]
        y_train = df['responder_6'].loc[df['date_id'].isin(selected_dates)]
        w_train = df['weight'].loc[df['date_id'].isin(selected_dates)]

        # Train the model based on the type (LightGBM, XGBoost, or CatBoost)
        if model_name == 'lgb':

            print("on est dans le cas lgb")
            # Train LightGBM model with early stopping and evaluation logging
            model.fit(X_train, y_train, w_train,  
                      eval_metric=[r2_lgb],
                      eval_set=[(X_valid, y_valid, w_valid)], 
                      callbacks=[
                          lgb.early_stopping(100), 
                          lgb.log_evaluation(10)
                      ])
            
        elif model_name == 'cbt':
            # Prepare evaluation set for CatBoost
            evalset = cbt.Pool(X_valid, y_valid, weight=w_valid)
            
            # Train CatBoost model with early stopping and verbose logging
            model.fit(X_train, y_train, sample_weight=w_train, 
                      eval_set=[evalset], 
                      verbose=10, 
                      early_stopping_rounds=100)
            
        else:
            # Train XGBoost model with early stopping and verbose logging
            model.fit(X_train, y_train, sample_weight=w_train, 
                      eval_set=[(X_valid, y_valid)], 
                      sample_weight_eval_set=[w_valid], 
                      verbose=10, 
                      early_stopping_rounds=100)

        # Append the trained model to the list
        models.append(model)
        
        # Save the trained model to a file
        joblib.dump(model, f'./models/{model_name}_{i}.model')

        print("fin du train")
        
        # Delete training data to free up memory
        del X_train
        del y_train
        del w_train
        
        # Collect garbage to free up memory
        import gc
        gc.collect()
        
    else:
        print("test")
        # If not in training mode, load the pre-trained model from the specified path
        models.append(joblib.load(f'{model_path}/{model_name}_{i}.model'))
        
    return 

# Custom R2 metric for XGBoost
def r2_xgb(y_true, y_pred, sample_weight):
    r2 = 1 - np.average((y_pred - y_true) ** 2, weights=sample_weight) / (np.average((y_true) ** 2, weights=sample_weight) + 1e-38)
    return -r2

# Custom R2 metric for LightGBM
def r2_lgb(y_true, y_pred, sample_weight):
    r2 = 1 - np.average((y_pred - y_true) ** 2, weights=sample_weight) / (np.average((y_true) ** 2, weights=sample_weight) + 1e-38)
    return 'r2', r2, True

# Custom R2 metric for CatBoost
class r2_cbt(object):
    def get_final_error(self, error, weight):
        return 1 - error / (weight + 1e-38)

    def is_max_optimal(self):
        return True

    def evaluate(self, approxes, target, weight):
        assert len(approxes) == 1
        assert len(target) == len(approxes[0])

        approx = approxes[0]

        error_sum = 0.0
        weight_sum = 0.0

        for i in range(len(approx)):
            w = 1.0 if weight is None else weight[i]
            weight_sum += w * (target[i] ** 2)
            error_sum += w * ((approx[i] - target[i]) ** 2)

        return error_sum, weight_sum

# Dictionary to store different models with their configurations

# premier test : (met bcp trop de temps à s'executer)
# model_dict = {
#     'lgb': lgb.LGBMRegressor(n_estimators=500, device='gpu', gpu_use_dp=True, objective='l2'),
#     'xgb': xgb.XGBRegressor(n_estimators=2000, learning_rate=0.1, max_depth=6, tree_method='hist', device="cuda", objective='reg:squarederror', eval_metric=r2_xgb, disable_default_eval_metric=True),
#     'cbt': cbt.CatBoostRegressor(iterations=1000, learning_rate=0.05, task_type='GPU', loss_function='RMSE', eval_metric=r2_cbt()),
# }  


# model_dict = {
#     'lgb': lgb.LGBMRegressor(n_estimators=20, device='gpu', gpu_use_dp=True, objective='l2', max_depth=5, num_leaves=20),
#     'xgb': xgb.XGBRegressor(  #si aucun gpu n'est disponible
#         n_estimators=50, 
#         learning_rate=0.2, 
#         max_depth=4, 
#         tree_method='hist',  # CPU-friendly option
#         objective='reg:squarederror', 
#         eval_metric=r2_xgb, 
#         disable_default_eval_metric=True),
#     #'xgb': xgb.XGBRegressor(n_estimators=50, learning_rate=0.2, max_depth=4, tree_method='gpu_hist', device="cuda", objective='reg:squarederror', eval_metric=r2_xgb, disable_default_eval_metric=True), #Si un gpu est disponible
#     'cbt': cbt.CatBoostRegressor(iterations=50, learning_rate=0.1, depth=6, task_type='GPU', loss_function='RMSE', eval_metric=r2_cbt()),
# }


# Train models for each fold


## Paramètres des modèles

In [12]:
model_dict = {
    # LightGBM sans GPU
    'lgb': lgb.LGBMRegressor(
        n_estimators=20,
        device='cpu',  # CPU mode
        objective='l2',
        max_depth=5,
        num_leaves=20
    ),
    # Version GPU de LightGBM (commentée)
    # 'lgb': lgb.LGBMRegressor(
    #     n_estimators=20,
    #     device='gpu',  # GPU mode
    #     gpu_use_dp=True,
    #     objective='l2',
    #     max_depth=5,
    #     num_leaves=20
    # ),
    
    # XGBoost sans GPU
    'xgb': xgb.XGBRegressor(  
        n_estimators=50, 
        learning_rate=0.2, 
        max_depth=4, 
        tree_method='hist',  # CPU-friendly option
        objective='reg:squarederror', 
        eval_metric=r2_xgb, 
        disable_default_eval_metric=True
    ),
    # Version GPU de XGBoost (commentée)
    # 'xgb': xgb.XGBRegressor(
    #     n_estimators=50,
    #     learning_rate=0.2,
    #     max_depth=4,
    #     tree_method='gpu_hist',  # GPU-friendly option
    #     objective='reg:squarederror',
    #     eval_metric=r2_xgb,
    #     disable_default_eval_metric=True
    # ),
    
    # CatBoost sans GPU
    'cbt': cbt.CatBoostRegressor(
        iterations=50,
        learning_rate=0.1,
        depth=6,
        task_type='CPU',  # CPU mode
        loss_function='RMSE',
        eval_metric=r2_cbt()
    ),
    # Version GPU de CatBoost (commentée)
    # 'cbt': cbt.CatBoostRegressor(
    #     iterations=50,
    #     learning_rate=0.1,
    #     depth=6,
    #     task_type='GPU',  # GPU mode
    #     loss_function='RMSE',
    #     eval_metric=r2_cbt()
    # ),
}


## Execution de l'entrainement

In [ ]:

for i in range(N_fold):
    train(model_dict, 'lgb')  #LGB prend bcp trop de temps à s'exécuter
    print('lgb done')
    train(model_dict, 'xgb')
    print('xgb done')
    train(model_dict, 'cbt')
    print('cbt done')

#### There seems to be bug in official code, can only submit polars dataframe

In [15]:
lags_ : pl.DataFrame | None = None

# Replace this function with your inference code.
# You can return either a Pandas or Polars dataframe, though Polars is recommended.
# Each batch of predictions (except the very first) must be returned within 10 minutes of the batch features being provided.
def predict(test: pl.DataFrame, lags: pl.DataFrame | None) -> pl.DataFrame | pd.DataFrame:
    """Make a prediction."""
    # All the responders from the previous day are passed in at time_id == 0. We save them in a global variable for access at every time_id.
    # Use them as extra features, if you like.
    global lags_
    if lags is not None:
        lags_ = lags

    predictions = test.select(
        'row_id',
        pl.lit(0.0).alias('responder_6'),
    )
    
    feat = test[feature_names].to_numpy()
    
    pred = [model.predict(feat) for model in models]
    pred = np.mean(pred, axis=0)
    
    predictions = predictions.with_columns(pl.Series('responder_6', pred.ravel()))

    # The predict function must return a DataFrame
    assert isinstance(predictions, pl.DataFrame | pd.DataFrame)
    # with columns 'row_id', 'responer_6'
    assert list(predictions.columns) == ['row_id', 'responder_6']
    # and as many rows as the test data.
    assert len(predictions) == len(test)

    return predictions

When your notebook is run on the hidden test set, inference_server.serve must be called within 15 minutes of the notebook starting or the gateway will throw an error. If you need more than 15 minutes to load your model you can do so during the very first `predict` call, which does not have the usual 10 minute response deadline.

In [ ]:
# !pip install kaggle-environments
# import kaggle_evaluation.jane_street_inference_server

In [ ]:
# inference_server = kaggle_evaluation.jane_street_inference_server.JSInferenceServer(predict)

# if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
#     inference_server.serve()
# else:
#     inference_server.run_local_gateway(
#         (
#             '/kaggle/input/jane-street-realtime-marketdata-forecasting/test.parquet',
#             '/kaggle/input/jane-street-realtime-marketdata-forecasting/lags.parquet',
#             # '/test.parquet',
#             # '/lags.parquet',
#         )
#     )

In [ ]:

# Import required module for the inference server
from kaggle_evaluation.jane_street_inference_server import JSInferenceServer

# Setting up the inference server with the predict function
inference_server = JSInferenceServer(predict)

# Start the server to handle submissions
if os.getenv("KAGGLE_IS_COMPETITION_RERUN"):
    inference_server.serve()
else:
    # For local testing (adjust paths as needed)
    inference_server.run_local_gateway(
        (
            '/kaggle/input/jane-street-realtime-marketdata-forecasting/test.parquet',
            '/kaggle/input/jane-street-realtime-marketdata-forecasting/lags.parquet',
        )
    )


In [ ]:

# Import required module for the inference server
from kaggle_evaluation.jane_street_inference_server import JSInferenceServer

# Setting up the inference server with the predict function
inference_server = JSInferenceServer(predict)

# Start the server to handle submissions
if os.getenv("KAGGLE_IS_COMPETITION_RERUN"):
    inference_server.serve()
else:
    # For local testing (adjust paths as needed)
    inference_server.run_local_gateway(
        (
            '/kaggle/input/jane-street-realtime-marketdata-forecasting/test.parquet',
            '/kaggle/input/jane-street-realtime-marketdata-forecasting/lags.parquet',
        )
    )
